In [108]:
import os
import re
import string
import glob
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [109]:
text_dir = "../data/raw/text"
labels_dir =  "../data/raw/labels"
audio_dir = "../data/raw/audio"



In [110]:
text_files = glob.glob(f"{text_dir}/*.txt")


def remove_stamps_str(line)->str:
    #clip_num = re.search('.+___\d\d?\d?___.+', line).group(0)
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

In [111]:
#Retire chaque stamps, chaque texte devien 1 seul str

def text_list_generator(files_list):
    text_list = []
    for filename in files_list:
        with open(file = filename, encoding = 'utf-8') as f:

            ##WINDOWS SPECIFIC
            videoid = filename.removeprefix(text_dir + '\\').rstrip('.txt')
            lines = f.readlines()
            for line_number, text_line in enumerate(lines):
                clean_line = remove_stamps_str(text_line)
                clip_id = videoid +'_' +str(line_number)
                yield (clip_id, clean_line.rstrip())

"""
corpus = []
text = []
text_list = []
for filename in text_files:
    text_str = ''
    with open(file=filename, encoding='utf-8') as f:
        lines = f.readlines()
        for count, line in enumerate(lines):
            clean_line = remove_stamps_str(line)
            text.append(clean_line)
    text_list.append(text)
    """

"\ncorpus = []\ntext = []\ntext_list = []\nfor filename in text_files:\n    text_str = ''\n    with open(file=filename, encoding='utf-8') as f:\n        lines = f.readlines()\n        for count, line in enumerate(lines):\n            clean_line = remove_stamps_str(line)\n            text.append(clean_line)\n    text_list.append(text)\n    "

In [112]:
stop_words = stopwords.words('English')

#création d'un dict pour lookup en O(1)
stopwords_dict = Counter(stop_words)

"""
def avg_count(l1:list):
    word_count = 0
    for e in l1:
        word_count += len(e.split())
    return int(word_count/len(l1))
"""


#Retire tous les timestamps en début de ligne, présents dans chaque transcript
def remove_stamps_str(line)->str:
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

#Retire les charactères non-ascii 
def remove_nonascii(line)->str:
    ascii_line = line.encode(encoding = 'ascii', errors = 'ignore').decode()
    return ascii_line

#met tout en minuscules, retire les nombres et stopwords
def clean_stopwords_digits(line)->str:
    new_line = ' '.join([word.lower() for word in line.split() if (len(word) >=2 and word.isalpha() and word not in stopwords_dict)])
    return new_line



In [113]:
label_df = pd.read_csv("../data/interim/labels/interim.csv")
display(label_df)

,id,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,0,0,0,1,0,0,1
1,-3g5yACwYnA_10,0,0,0,0,0,0,1
2,-3g5yACwYnA_13,0,0,0,0,0,0,1
3,-3g5yACwYnA_2,0,0,0,1,0,0,0
4,-3g5yACwYnA_3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
21860,zwTrXwi54us_6,0,0,0,0,0,0,0
21861,zwTrXwi54us_7,0,0,0,0,0,0,0
21862,zwTrXwi54us_8,0,0,0,0,0,0,0
21863,zwTrXwi54us_9,0,0,0,0,0,0,1


In [114]:
corpus = (text for text in text_list_generator(text_files))
df_text = pd.DataFrame(corpus)
df_text.columns = ['id', 'text']

In [115]:
display(df_text.head(5))

,id,text
0,--qXJuDtHPw_0,I see that there are three category of writers.
1,--qXJuDtHPw_1,"I define them as being an author, a writer, an..."
2,--qXJuDtHPw_2,"An author, I like to classify as somebody who ..."
3,--qXJuDtHPw_3,These are the well-known authors of our time a...
4,--qXJuDtHPw_4,"Then, there is the writer."


In [116]:

df_text['text'] =  df_text.text.apply(remove_nonascii)





#df_text['tokens'] = df_text['pre_cleaned_text'].apply(lambda l : word_tokenize(''.join(clean_stopwords_digits(l))))


#df_text['lines'] = df_text.groupby('unique_id')
df_text['count'] = df_text['text'].apply(lambda x : len(x.split(' ')))
#df_text['word_count'] = df_text['tokens'].apply(len)



display(df_text)

,id,text,count
0,--qXJuDtHPw_0,I see that there are three category of writers.,9
1,--qXJuDtHPw_1,"I define them as being an author, a writer, an...",13
2,--qXJuDtHPw_2,"An author, I like to classify as somebody who ...",14
3,--qXJuDtHPw_3,These are the well-known authors of our time a...,11
4,--qXJuDtHPw_4,"Then, there is the writer.",5
...,...,...,...
44972,_ZlF5Q9W1DM_1,Clearly because everything is data driven toda...,17
44973,_ZlF5Q9W1DM_2,"But the conceptual drivers, what sort of drive...",28
44974,_ZlF5Q9W1DM_3,So those two things actually I think rub up ag...,21
44975,_ZlF5Q9W1DM_4,So the idea of numbers telling the story up ag...,40


In [117]:
unique_labels = set(label_df.id)
unique_labels


{'sqa_mosi/AHiA9hohKr8_2',
 'omHQ68k50XY_14',
 '261900_27',
 'omBcvw7izuM_19',
 '9cxlcpbmrH0_6',
 'sqa_mosi/NCJTV5KaJJc_6',
 '244261_2',
 's1Yo_MCiMPc_16',
 'mpfSRGHFY0g_1',
 '4jTUcNlxlYA_7',
 'pom_extra/81538_7',
 'd7Dz34-cZ74_13',
 'IHp8hd1jm6k_4',
 'sqa_mosi/KRje7su4I5U_5',
 '_gzYkdjNvPc_5',
 'xwNQ7QuXaps_9',
 '243981_15',
 '224370_12',
 'pom_extra/136215_2',
 '29751_2',
 'GAVpYuhMZAw_6',
 'sqa_mosi/mHIvH6Nnrls_3',
 'pom_extra/56006_7',
 '244261_7',
 '90wZz0thye4_6',
 'uu1FqsR7JV0_1',
 'AepwIhSbAws_2',
 'j1m6ctAgjsM_30',
 '9BceQC_SdTM_1',
 'pom_extra/100178_1',
 'z441aDJvAcU_25',
 '190740_1',
 'sqa_mosi/kqXpf26EL-s_3',
 'jrDqduyQrfo_1',
 'sqa_mosi/fU5AYkq0m9k_4',
 '-Alixo7euuU_0',
 'pom_extra/180971_0',
 'i714Xt6eb1Y_5',
 'sqa_mosi/LcfubBagG6Q_6',
 '22373_2',
 'pom_extra/238683_4',
 '277991_1',
 'CvW0Mh1EWFA_4',
 '4WTVL5bNybU_0',
 '50103_3',
 '73447_1',
 'xu53lk4dhSc_24',
 'ROC2YI3tDsk_5',
 '266396_58',
 'F59hwsm4Ld0_31',
 'sqa_mosi/pOrk7fl1AYo_4',
 'oaC1UATdtAg_4',
 'sqa_mosi/mxa4K

In [118]:
frame = pd.merge(df_text, label_df, on = 'id', how = 'inner')
frame

,id,text,count,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,I see that a writer is somebody who has an inc...,18,0,0,0,1,0,0,1
1,-3g5yACwYnA_2,Key Polymer brings a technical aspect to our o...,14,0,0,0,1,0,0,0
2,-3g5yACwYnA_3,We're a huge user of adhesives for our operati...,29,0,0,0,0,0,0,0
3,-3g5yACwYnA_4,Key brings those types of aspects to a busines...,34,0,0,0,0,0,0,0
4,-3g5yACwYnA_9,We have many new opportunities through the way...,21,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
15384,_xjBqNfiEY4_6,"And once again, U of I students are so well pr...",24,0,0,0,0,0,0,0
15385,_XWUKMkxa-Q_4,So its become more of an iterative process wit...,13,0,0,0,1,0,0,1
15386,_YzIvwvyIq4_5,"Secondly, using social, you know, things like ...",30,0,0,0,1,0,0,1
15387,_ZlF5Q9W1DM_0,JOHN GERZEMA: When I think about finding insig...,24,0,0,0,0,0,0,0


In [119]:
nltk.download('averaged_perceptron_tagger')
df_text['tagged'] = df_text['tokens'].apply(nltk.tag.pos_tag )

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\trott\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


KeyError: 'tokens'

Bag of words

In [ ]:
import gensim.downloader as api

w2v_model = api.load("word2vec-google-news-300")

#SEULEMENT PRENDRE MOT DANS MODEL.VOCAB